# Módulo 6 Actividad 4

## Intención del aprendizaje esperado:


**4. Elabora un modelo predictivo utilizando técnicas de reducción dimensional para resolver un problema de aprendizaje de máquina**

Ejercicios Planteados

La base de datos House_price.csv contiene información de casas. La idea es obtener un modelo que permita predecir el valor de venta de una casa. Las variables son las siguientes:

• LotFrontage: Pies lineales de calle conectados a la propiedad

• LotArea: Tamaño del lote en pies cuadrados

• MasVnrArea: Área de revestimiento de mampostería en pies cuadrados

• BsmtFinSF1: Pies cuadrados del sótano terminados tipo 1

• BsmtFinSF2: Pies cuadrados con del sótano acabados tipo 2

• BsmtUnfSF: Pies cuadrados sin terminar de área del sótano

• TotalBsmtSF: Total de pies cuadrados de área del sótano

• 1stFlrSF: Pies cuadrados del primer piso

• 2ndFlrSF: Pies cuadrados del segundo piso

• LowQualFinSF: Pies cuadrados con acabado de baja calidad (todos los pisos)

• GrLivArea: Pies cuadrados de área habitable sobre el nivel (suelo)

• GarageArea: Tamaño del garaje en pies cuadrados

• WoodDeckSF: Área de la plataforma de madera en pies cuadrados

• OpenPorchSF: Área del porche abierto en pies cuadrados

• EnclosedPorch: Área del porche cerrado en pies cuadrados

• 3SsnPorch: área de porche de tres estaciones en pies cuadrados

• ScreenPorch: Área del porche de la pantalla en pies cuadrados

• PoolArea: Área de la piscina en pies cuadrados

• MiscVal: Valor de la función miscelánea

• SalePrice: el precio de venta de la propiedad en dólares. Esta es la variable objetivo que se quiere modelar.


**a) Cargue la base de datos, ¿hay alguna columna que no sea útil para el análisis? Revise si existen casos faltantes en la base de datos, según la descripción de las variables, ¿qué pudiera significar un NA? Tome decisiones de qué hacer si existen casos faltantes. Si desea imputar los valores nulos de alguna variable por un valor puede utilizar data["columna"].fillna('valor', inplace = True).**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:


precios_house = pd.read_csv('house_price.csv', delimiter=';')

pd.set_option("display.max_columns", None)

precios_house.head(3)

Se realiza un repaso por la información contenida en el df para contextualizar la problemática.

- información del df

- columnas con valores Nan

- medidas de tendencia central, cuartiles y mínimos y máximos de las columnas

In [ ]:
precios_house.info()

conteo_registros_nan = precios_house.isna().sum()
columnas_con_nan = []
rango_itera = precios_house.shape[1]
range(rango_itera)

pd.reset_option('display.max_columns')
pd.set_option('display.max_rows', None)

for i in range(rango_itera):
    if conteo_registros_nan.iloc[i] != 0:
        columnas_con_nan.append(conteo_registros_nan.index[i])

print('\n' + 'COLUMNAS CON REGISTROS NaN '.center(50, '=') + '\n')

for i in range(len(columnas_con_nan)):
    print('- ',columnas_con_nan[i] + '\n')



print(' REPORTE MEDIDAS DE TENDENCIA COLUMNAS CON NaN '.center(50, '=') + '\n')
print(precios_house[columnas_con_nan].describe())

pd.reset_option('display.max_rows')

In [ ]:
media_lotFrontage = precios_house['LotFrontage'].mean()
media_MasVnrArea = precios_house['MasVnrArea'].mean()

precios_house['LotFrontage'] = precios_house['LotFrontage'].fillna(media_lotFrontage)
precios_house['MasVnrArea'] = precios_house['MasVnrArea'].fillna(media_MasVnrArea)


Decido completar los valores Nan del df con el valor de la media de sus elementos.  En el caso de la columnas LotFrontage son 259 valores faltantes, se estudian sus medidas de tendencia y bajo ese criterio se procede a completar con la media de la columna pues me parece que es valor que mejor representa el grupo con los pocos datos que tenemos y el conocimiento técnico sobre el área de bienes raíces.

**b) La variable a modelar corresponde al precio de venta de la casa en dólares. En base a los modelos que hemos aprendido en clases, ¿qué modelo(s) podría(n) ser de utilidad para predecir el precio de venta de casas mediante las variables predictoras?**

Debido al objetivo del problema, que tiene que ver más con una predicción que con una clasificación, debiésemos optar por un modelo de regresión lineal.  Si bien, existen modelos de clasificación que permiten la predicción de resultados, el modelo de regresión lineal tiene muchas ventajas de rendimiento respecto a estos otros estimadores.

**c) Vamos a implementar un modelo de regresión lineal para modelar el precio de venta de casas. Considerando que en la base de datos tenemos varias variables que dependen de otras, ¿cuál pudiera ser el problema que surgiría en este contexto? ¿por qué esto es un problema? Discuta.**

Para mejorar el rendimiento de un modelo de predicción es importante evaluar la relevancia de cada variable respecto al resultado. Según el enunciado en la base se tiene varias variables que dependen de otras lo que puede traer como consecuencia multicolinealidad lo que implica que existen variables independientes que están altamente correlacionadas entre sí.  Esto podría derivar en diversos problemas como la *inestabilidad de los coeficientes estimados* para variables independientes altamente correlacionadas se pierde la objetividad para estimar cuanto influye cada una de las variables de manera independiente siendo propensa a inexactitudes y sensibilidad a variaciones en los datos y por consiguiente la interpretación errónea de los coeficientes.  La multicolinealidad influye por lo mismo de manera negativa en la *precision de las predicciones* producto del ruido introducido por las variables correlacionadas y la dificultad de evaluar su verdadera influencia particular obteniendo resultados menos precisos.  También redunda en la dificultad de poder reconocer la verdadera influencia de cada variable en particular y por lo mismo perder la posibilidad de reconocer las variables más importantes.

Existen maneras de enfrentar la multicolinealidad como la eliminación de variables redundantes, la transformación de los datos de la variable mediante operaciones matemáticas que podrían cambiar el comportamiento en otras magnitudes, métodos de regularización y el PCA (Análisis de Componentes Principales)

**d) Realice un análisis de la correlación entre las variables. Comente, ¿existen variables muy correlacionadas entre sí? ¿Por qué cree que ocurre?**


In [ ]:
correlacion = precios_house.corr()
columnas_altamente_correlacionadas = ['TotalBsmtSF', '1stFlrSF']


plt.figure(figsize=(15, 10))
sns.heatmap(correlacion, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.title('Matriz de correlación')
plt.show()


fig, axes = plt.subplot_mosaic([['upleft', 'right'], ['lowleft', 'right']], layout='constrained', figsize=(12, 5))

sns.scatterplot(ax=axes['right'], x='TotalBsmtSF', y='1stFlrSF', data=precios_house)
axes['right'].set_title('Scatterplot entre TotalBsmtSF y 1stFlrSF')
sns.boxplot(ax=axes['upleft'], x=precios_house['TotalBsmtSF'])
axes['upleft'].set_title('Boxplot de la TotalBsmtSF')
sns.boxplot(ax=axes['lowleft'], x=precios_house['1stFlrSF'])
axes['lowleft'].set_title('Boxplot de la 1stFlrSF')
axes['upleft'].set_xlim(-50, 7000)
axes['lowleft'].set_xlim(-50, 7000)
plt.show()

print(precios_house[columnas_altamente_correlacionadas].describe())



De las 21 variables, 20 restando la variable target, podemos ver que en general existe bajo índice de correlación entre ellas e, incluso, de la mayoría de las variables respecto a la variable target, sin embargo llama la atención en la matriz de correlación que las variables 'TotalBsmtSF' y '1stFlrSF' tienen el mismo índice de correlación con la variable target (61% lo que es un valor considerable) y además entre ellas también tienen una correlación alta, la más alta entre todas las variables.  Esta situación implica problemas de colinealidad que debemos abordar en detalle para poder tomar una decisión.

Se generan 3 gráficos para visualizar ciertos parámetros.  Los gráficos de cajas nos permiten observar que el comportamiento en general es similar, con la excepción de que 'TotalBsmtSF' tiene registros de valor 0 que los posicionan fuera del bigote izquierdo incluso, pero con esta excepción ya comentada, el comportamiento de los gráficos es bastante similar, con una mediana ubicada en el mismo lugar guardando las proporciones, lo mismo con la distribución de los bigotes y sus eventos a la derecha del bigote.  Un tercer gráfico nos permite visualizar en un plano la distribución los eventos bajo las dos variables mencionadas y observar que efectivamente existe una tendencia a que los eventos coincidan en sus coordenadas (x=y) sin embargo también podemos observar que existen registros con valores 0, que perfectamente nos puede permitir inferir que 0 es la moda de la columna 'TotalBsmtSF', y que la mayor concentración de los eventos se genera entre el cuadrante generado por x=2000, y=2000.

Respecto al motivo de la colinealidad, no me parece tan ilógico que cuando las casas cuenten con un sótano este sea en tamaño proporcional a las dimensiones del primer piso.  Una manera de afrontar esta colinealidad podría ser cambiar el enfoque de la variable 'TotalBsmtSF' a un enfoque booleano de registros == 0 es False y != 0 es True y evaluar el impacto de la variable no mediante su valor escalar sino que mediante el enfoque booleano, dejando a '1stFlrSF' en valores originales ya que que la casa tenga primer piso es lo primordial o prima sobre 'TotalBsmtSF'.

**e) Suponga que este es un problema de gran volumen, donde tenemos muchas columnas y registros. Cuando existen variables muy correlacionadas el costo computacional para obtener el modelo puede ser muy elevado. Obtenga el tiempo de procesamiento para obtener una regresión lineal. Para esto, podemos obtener la regresión lineal utilizando la función LinearRegression de sklearn.linear_model y calcular el tiempo de procesamiento de la siguiente forma:**


from datetime import datetime

start=datetime.now()

[aquí va el código del modelo]

print(datetime.now()-start) #esto entrega el tiempo que demoró en correr el modelo.


In [ ]:

X = precios_house.drop(['Id', 'SalePrice'], axis=1)
y = precios_house['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

estimador_regresion_lineal = LinearRegression()

start = datetime.now()

estimador_regresion_lineal.fit(X_train, y_train)
y_pred = estimador_regresion_lineal.predict(X_test)

print(datetime.now() - start)




**f) Obtenga las componentes principales utilizando las variables estandarizadas, si desea explicar un 80% de la variabilidad, ¿cuántas componentes debería elegir? Argumente.**

**g) Corra el modelo de regresión lineal con las componentes principales como variables predictoras. Vuelva a calcular el tiempo de procesamiento. ¿En cuál caso suele demorarse menos? ¿Qué ganamos al realizar componentes principales? Concluya.**


In [ ]:

nombres_columnas = [X.columns]

scaler = StandardScaler()
X_estandarizados = scaler.fit_transform(X)

pca = PCA(n_components=0.8)

componentes_principales = pca.fit_transform(X_estandarizados)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(componentes_principales, y, test_size=0.3, random_state=42)

estimador_2 = LinearRegression()

start1 = datetime.now()

estimador_2.fit(X_train_2, y_train)
y_pred_2 = estimador_2.predict(X_test_2)

print(datetime.now() - start1)


Utilizamos la notación 'PCA(n_componentes=0.8)' con la cual instruímos a la máquina a obtener los mejores PCA que expliquen el 80% de la varianza acumulada.  En cuanto al rendimiento, el modelo funciona más rápido cuando aplicamos el método de PCA pero la diferencia es tan pequeña que no es relevante al menos para las dimensiones de este set de datos.